In [183]:
import pandas as pd
import numpy as np

import os
from tqdm import tqdm
import pickle
pd.set_option('display.max_columns', None)

# Split Data

In [378]:
from openpyxl import load_workbook
import pandas as pd
workbook = load_workbook('./raw/LK_modified.xlsx')
sheet_names = workbook.sheetnames

In [379]:
data = []
sheet = workbook[sheet_names[1]]
for row in sheet.iter_rows(values_only=True):
    data.append(row)
df_qa = pd.DataFrame(data[1:], columns=data[0])
df_qa.to_csv('data_qa.csv')

In [380]:
data = []
sheet = workbook[sheet_names[2]]
for row in sheet.iter_rows(values_only=True):
    data.append(row)
df_glossary = pd.DataFrame(data[1:], columns=data[0])
df_glossary.to_csv('data_glossary.csv')

In [381]:
data = []
sheet = workbook[sheet_names[0]]
for row in sheet.iter_rows(values_only=True):
    data.append(row)
df_qa_popular = pd.DataFrame(data[1:], columns=data[0])
df_qa_popular.to_csv('data_qa_popular.csv')

# Glossary

In [382]:
df_glossary

,Сокращение,Расшифровка
0,лк,личный кабинет
1,БиР,Беременность и роды
2,зп,заработная плата
3,НДФЛ,Налог на доходы физических лиц
4,СТД,срочный трудовой договор
5,ТК,трудовой договор
6,АО,авансовый отчет
7,SLA,сроки
8,ЭЦП,электронная цифровая подпись
9,КР,кадровый резерв


In [383]:
df_glossary['Расшифровка'] = df_glossary['Расшифровка'].str.lower()
df_glossary['Сокращение'] = df_glossary['Сокращение'].str.lower()

In [384]:
dict_glossary = pd.Series(df_glossary['Расшифровка'].values,index=df_glossary['Сокращение']).to_dict()

In [385]:
# АТЗ, 790 - 793 доп отпуска ЧАЭЕ,МКС,РКС, НРД, 832 БС, 907 СОТ?, 988 сф бп,1501 дс - доп соглашение
dict_glossary['тк'] = 'трудовая книжка'
# dict_glossary['тк рф'] = 'трудовой кодекс российской федерации'
dict_glossary['тк рф'] = 'трудовой кодекс'
dict_glossary['рф'] = 'российская федерация'
dict_glossary['мчд'] = 'машиночитаемая доверенность'
dict_glossary['ур'] = 'удалённая работа'
# dict_glossary['2ндфл'] =
dict_glossary['дмс'] = 'добровольное медицинское страхование'
dict_glossary['дм'] = 'директор магазина'
dict_glossary['дмп'] = 'директор магазина партнёр'
dict_glossary['здм'] = 'заместитель директора магазина'
dict_glossary['адм'] = 'ассистент директора магазина'
dict_glossary['есп'] = 'единая система подбора'
dict_glossary['эп'] = 'электронная подпись'
dict_glossary['зп'] = 'зарплата'
dict_glossary['з/п'] = 'зарплата'
dict_glossary['дк'] = 'директор кластеров'
dict_glossary['мп'] = 'мобильное приложение'
dict_glossary['кр'] = 'кадровый резерв'
dict_glossary['lk'] = 'личный кабинет'
dict_glossary['бл'] = 'больничный лист'
dict_glossary['бс'] = 'отпуск без содержания'
dict_glossary['укэп'] = 'усиленная квалифицированная электронная подпись'
dict_glossary['унэп'] = 'усиленная неквалифицированная электронная подпись'
dict_glossary['фл'] = 'физическое лицо'
dict_glossary

{'лк': 'личный кабинет',
 'бир': 'беременность и роды',
 'зп': 'зарплата',
 'ндфл': 'налог на доходы физических лиц',
 'стд': 'срочный трудовой договор',
 'тк': 'трудовая книжка',
 'ао': 'авансовый отчет',
 'sla': 'сроки',
 'эцп': 'электронная цифровая подпись',
 'кр': 'кадровый резерв',
 'тк рф': 'трудовой кодекс',
 'рф': 'российская федерация',
 'мчд': 'машиночитаемая доверенность',
 'ур': 'удалённая работа',
 'дмс': 'добровольное медицинское страхование',
 'дм': 'директор магазина',
 'дмп': 'директор магазина партнёр',
 'здм': 'заместитель директора магазина',
 'адм': 'ассистент директора магазина',
 'есп': 'единая система подбора',
 'эп': 'электронная подпись',
 'з/п': 'зарплата',
 'дк': 'директор кластеров',
 'мп': 'мобильное приложение',
 'lk': 'личный кабинет',
 'бл': 'больничный лист',
 'бс': 'отпуск без содержания',
 'укэп': 'усиленная квалифицированная электронная подпись',
 'унэп': 'усиленная неквалифицированная электронная подпись',
 'фл': 'физическое лицо'}

# Processing

In [386]:
from string import punctuation
punctuation_marks = set(punctuation)

glossary = dict_glossary.keys()
glossary

dict_keys(['лк', 'бир', 'зп', 'ндфл', 'стд', 'тк', 'ао', 'sla', 'эцп', 'кр', 'тк рф', 'рф', 'мчд', 'ур', 'дмс', 'дм', 'дмп', 'здм', 'адм', 'есп', 'эп', 'з/п', 'дк', 'мп', 'lk', 'бл', 'бс', 'укэп', 'унэп', 'фл'])

In [387]:
df_qa['question'] = df_qa['question'].str.lower()
df_qa['content'] = df_qa['content'].str.lower()
df_qa['category'] = df_qa['category'].str.lower()

In [388]:
def replace_abbr(content_list):
    content_list_changed=[]
    for content in content_list:
        content_list = content.lower().split()
        for ind, word in enumerate(content_list):
            sign = ''
            if(word[-1] in punctuation_marks):
                sign = word[-1]
                word = word[:-1:]
            if word in glossary:   
                if word == 'тк':
                    word2 = content_list[ind+1]
                    if word2[-1] in punctuation_marks:
                        sign = word2[-1]
                        word2 = word2[:-1:]
                    if(word2 == 'рф'):
                        content_list[ind] = dict_glossary['тк рф']
                        continue    
                content_list[ind] =  dict_glossary[word] + sign
        content_list_changed.append(" ".join(content_list))
    return content_list_changed

In [389]:
df_qa['question_changed'] = replace_abbr(df_qa.question.to_list())
df_qa['content_changed'] =  replace_abbr(df_qa.content.to_list())
df_qa['category_changed'] = replace_abbr(df_qa.category.to_list())

In [390]:
df_qa.sample(5)

,None,question,content,category,question_changed,content_changed,category_changed
951,967,3 мая сотруднику был назначен тест по текущей ...,"создайте, пожалуйста, обращение в ит поддержку...",поддержка,3 мая сотруднику был назначен тест по текущей ...,"создайте, пожалуйста, обращение в ит поддержку...",поддержка
1030,1046,не формируются документы на отпуск без сохране...,"создайте, пожалуйста, обращение в ит поддержку...",поддержка,не формируются документы на отпуск без сохране...,"создайте, пожалуйста, обращение в ит поддержку...",поддержка
788,803,как оформить доп. дни отпуска за сдачу крови,"при сдаче крови и её компонентов, предоставляе...",отпуск,как оформить доп. дни отпуска за сдачу крови,"при сдаче крови и её компонентов, предоставляе...",отпуск
38,38,нужно провести отпуск беременной сотруднице,для оформления ежегодного отпуска вне графика ...,бир,нужно провести отпуск беременной сотруднице,для оформления ежегодного отпуска вне графика ...,беременность и роды
693,706,у сотрудника нет в личном кабинете кнопки на о...,"доступ к программе ""карьера"" появляется спустя...",моя карьера,у сотрудника нет в личном кабинете кнопки на о...,"доступ к программе ""карьера"" появляется спустя...",моя карьера


In [391]:
# q_list = df_qa.question.to_list()
# q_list_changed=[]
# for question in q_list:
#     question_list = question.lower().split()
#     for ind, word in enumerate(question_list):
#         sign = ''
#         if(word[-1] in punctuation_marks):
#             sign = word[-1]
#             word = word[:-1:]
#         if word in glossary:
#             question_list[ind] =  dict_glossary[word] + sign
#     q_list_changed.append(" ".join(question_list))
# df_qa['question_changed'] = q_list_changed

# content_list = df_qa.content.to_list()
# content_list_changed=[]
# for content in content_list:
#     content_list = content.lower().split()
#     for ind, word in enumerate(content_list):
#         sign = ''
#         if(word[-1] in punctuation_marks):
#             sign = word[-1]
#             word = word[:-1:]
#         if word in glossary:
#             content_list[ind] =  dict_glossary[word] + sign
#     content_list_changed.append(" ".join(content_list))
# df_qa['content_changed'] = content_list_changed

# category_list = df_qa.category.to_list()
# category_list_changed=[]
# for category in category_list:
#     category_list = category.lower().split()
#     for ind, word in enumerate(category_list):
#         sign = ''
#         if(word[-1] in punctuation_marks):
#             word = word[:-1:]
#             sign = word[-1]
#         if word in glossary:
#             category_list[ind] =  dict_glossary[word] + sign
#     category_list_changed.append(" ".join(category_list))
# df_qa['category_changed'] = category_list_changed

In [392]:
df_qa.category.unique()

array(['автомобиль', 'бир', 'больничный', 'график работы', 'доверенность',
       'зарплата', 'заявки', 'командировка', 'лк', 'материальная помощь',
       'моя карьера', 'мчд', 'налоговый вычет', 'отгул', 'отпуск',
       'перевод', 'поддержка', 'эцп', 'документооборот',
       'прием на работу', 'сб', 'справка', 'табель', 'увольнение',
       'удаленная работа', 'уход за больным', 'sed', 'выручай-карта',
       'дмс', 'оператор', 'обучение'], dtype=object)

# Drop duplicates

In [393]:
df_qa = df_qa.drop_duplicates(subset=['question_changed','content_changed'], keep='last')
df_qa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1630 entries, 0 to 1675
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   None              1630 non-null   int64 
 1   question          1630 non-null   object
 2   content           1630 non-null   object
 3   category          1630 non-null   object
 4   question_changed  1630 non-null   object
 5   content_changed   1630 non-null   object
 6   category_changed  1630 non-null   object
dtypes: int64(1), object(6)
memory usage: 101.9+ KB


In [394]:
df_qa[df_qa.duplicated(subset=['question_changed'], keep=False)]

,None,question,content,category,question_changed,content_changed,category_changed
205,205,архивная задача вернулась,"если сотрудник архива не выполнил задача ""подт...",лк,архивная задача вернулась,"если сотрудник архива не выполнил задача ""подт...",личный кабинет
221,221,не могу закрыть заявку,"первым делом, просьба очистить кэш/куки браузе...",лк,не могу закрыть заявку,"первым делом, просьба очистить кэш/куки браузе...",личный кабинет
838,854,архивная задача вернулась,"есть причины, по которым сотрудник может не уч...",отпуск,архивная задача вернулась,"есть причины, по которым сотрудник может не уч...",отпуск
932,948,надбавка,"создайте, пожалуйста, обращение в ит поддержку...",поддержка,надбавка,"создайте, пожалуйста, обращение в ит поддержку...",поддержка
1105,1121,не могу закрыть заявку,"создайте, пожалуйста, обращение в ит поддержку...",поддержка,не могу закрыть заявку,"создайте, пожалуйста, обращение в ит поддержку...",поддержка
1664,1695,надбавка,по данному вопросу вы можете обратиться в кадр...,лк,надбавка,по данному вопросу вы можете обратиться в кадр...,личный кабинет


In [395]:
df_qa = df_qa[~df_qa[None].isin([854, 1121, 948])]

In [400]:
df_qa.sample(5)

,None,question,content,category,question_changed,content_changed,category_changed
881,897,"хочу поменять магазин, что делать?","создайте заявку по теме ""перевод"" в списке зая...",перевод,"хочу поменять магазин, что делать?","создайте заявку по теме ""перевод"" в списке зая...",перевод
764,779,если ежегодный отпуск совпал с отпуском по бер...,в случае пересечения ежегодного отпуска с отпу...,отпуск,если ежегодный отпуск совпал с отпуском по бер...,в случае пересечения ежегодного отпуска с отпу...,отпуск
1411,1430,здравствуйте могу ли я написать заявление об у...,"напишите заявление на увольнение , шаблон есть...",увольнение,здравствуйте могу ли я написать заявление об у...,"напишите заявление на увольнение , шаблон есть...",увольнение
100,100,где взять подпись на доверенность юриста,специалистами сф пб не производится подпись до...,доверенность,где взять подпись на доверенность юриста,специалистами сф пб не производится подпись до...,доверенность
24,24,возможно ли получить пособие по беременности и...,"если вы работаете в нескольких местах, то може...",бир,возможно ли получить пособие по беременности и...,"если вы работаете в нескольких местах, то може...",беременность и роды


# Catalogs

In [401]:
catalogs = {
    "здоровье": [
        "беременность и роды",
        "больничный",
        "справка",
        "уход за больным",
        "добровольное медицинское страхование"
    ],
    "аккаунт": [
        "заявки",
        "личный кабинет",
        "поддержка",
        "sed",
        "табель"
    ],
    "оператор": [
        "оператор",
    ],
    "финансы": [
        "зарплата",
        "материальная помощь",
        "налоговый вычет",
        "выручай-карта",
    ],
    "работа": [
        "график работы",
        "командировка",
        "моя карьера",
        "отгул",
        "отпуск",
        "перевод",
        "прием на работу",
        "увольнение",
        "удаленная работа",
        "обучение"
    ],
    "документы": [
        "доверенность",
        "машиночитаемая доверенность",
        "электронная цифровая подпись",
        "документооборот",
        "сб",
        "автомобиль"
    ]
}


In [402]:
reversed_catalogs = {}
for key, values in catalogs.items():
    for value in values:
        reversed_catalogs[value] = key
reversed_catalogs

{'беременность и роды': 'здоровье',
 'больничный': 'здоровье',
 'справка': 'здоровье',
 'уход за больным': 'здоровье',
 'добровольное медицинское страхование': 'здоровье',
 'заявки': 'аккаунт',
 'личный кабинет': 'аккаунт',
 'поддержка': 'аккаунт',
 'sed': 'аккаунт',
 'табель': 'аккаунт',
 'оператор': 'оператор',
 'зарплата': 'финансы',
 'материальная помощь': 'финансы',
 'налоговый вычет': 'финансы',
 'выручай-карта': 'финансы',
 'график работы': 'работа',
 'командировка': 'работа',
 'моя карьера': 'работа',
 'отгул': 'работа',
 'отпуск': 'работа',
 'перевод': 'работа',
 'прием на работу': 'работа',
 'увольнение': 'работа',
 'удаленная работа': 'работа',
 'обучение': 'работа',
 'доверенность': 'документы',
 'машиночитаемая доверенность': 'документы',
 'электронная цифровая подпись': 'документы',
 'документооборот': 'документы',
 'сб': 'документы',
 'автомобиль': 'документы'}

In [403]:
df_catalogs = pd.DataFrame(list(reversed_catalogs.items()), columns=['category_changed', 'catalog'])
if ('catalog' not in df_qa.columns):
    df_qa=df_qa.merge(df_catalogs,how='left', on='category_changed')
df_qa.sample(5)

,None,question,content,category,question_changed,content_changed,category_changed,catalog
436,468,нет пароля,"при проблемах со входом в личный кабинет, преж...",лк,нет пароля,"при проблемах со входом в личный кабинет, преж...",личный кабинет,аккаунт
981,1041,отсутствует ярлык отпуск без сохранения з/п,"создайте, пожалуйста, обращение в ит поддержку...",поддержка,отсутствует ярлык отпуск без сохранения зарплата,"создайте, пожалуйста, обращение в ит поддержку...",поддержка,аккаунт
8,8,"будет ли оплачен больничный, если он открыт в ...",сотруднику следует обратиться в мед. организац...,бир,"будет ли оплачен больничный, если он открыт в ...",сотруднику следует обратиться в мед. организац...,беременность и роды,здоровье
1293,1360,в лк нет доступа для заяврк на сотрудников маг...,"если в ""команде"" нет подчиненных сотрудников п...",табель,в личный кабинет нет доступа для заяврк на сот...,"если в ""команде"" нет подчиненных сотрудников п...",табель,аккаунт
338,340,не прогружаются данные на сотрудника,"если сотрудник трудоустроен сегодня, данные о ...",лк,не прогружаются данные на сотрудника,"если сотрудник трудоустроен сегодня, данные о ...",личный кабинет,аккаунт


In [404]:
df_qa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1627 entries, 0 to 1626
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   None              1627 non-null   int64 
 1   question          1627 non-null   object
 2   content           1627 non-null   object
 3   category          1627 non-null   object
 4   question_changed  1627 non-null   object
 5   content_changed   1627 non-null   object
 6   category_changed  1627 non-null   object
 7   catalog           1627 non-null   object
dtypes: int64(1), object(7)
memory usage: 101.8+ KB


In [405]:
df_qa.catalog.value_counts()

catalog
аккаунт      929
работа       467
здоровье      88
документы     72
финансы       66
оператор       5
Name: count, dtype: int64

In [406]:
df_qa[None].value_counts()

1652    3
0       1
1141    1
1151    1
1150    1
       ..
583     1
582     1
581     1
580     1
1706    1
Name: count, Length: 1625, dtype: int64

In [351]:
df_qa.loc[(df_qa.catalog == 'оператор')]

,None,question,content,category,question_changed,content_changed,category_changed,catalog
1583,1652,уйди,для оформления обращения в техническую поддерж...,оператор,уйди,для оформления обращения в техническую поддерж...,оператор,оператор
1584,1652,оператор,для оформления обращения в техническую поддерж...,оператор,оператор,для оформления обращения в техническую поддерж...,оператор,оператор
1585,1652,нужен оператор,для оформления обращения в техническую поддерж...,оператор,нужен оператор,для оформления обращения в техническую поддерж...,оператор,оператор
1587,1654,номер телефона службы поддержки,для оформления обращения в техническую поддерж...,оператор,номер телефона службы поддержки,для оформления обращения в техническую поддерж...,оператор,оператор
1611,1688,в какую службу поддержки обратиться по личному...,для оформления обращения в техническую поддерж...,оператор,в какую службу поддержки обратиться по личному...,для оформления обращения в техническую поддерж...,оператор,оператор


In [407]:
df_qa.category_changed.value_counts()

category_changed
личный кабинет                          444
поддержка                               271
табель                                  166
отпуск                                  119
удаленная работа                         98
увольнение                               60
моя карьера                              53
беременность и роды                      50
прием на работу                          48
отгул                                    48
заявки                                   47
зарплата                                 45
электронная цифровая подпись             33
график работы                            28
документооборот                          22
больничный                               17
налоговый вычет                          12
справка                                   9
уход за больным                           9
материальная помощь                       8
автомобиль                                8
доверенность                              7
обучение       

In [408]:
df_qa[df_qa.category == 'лк'].content.value_counts()[:10:]

content
при проблемах со входом в личный кабинет, прежде чем создавать заявку в поддержку, убедитесь, что заходите в лк на сайте https://company-x5.ru, указываете актуальные и верные логин и пароль. если вам неизвестен логин, обратитесь к руководителю (дм), он сможет посмотреть ваш логин и сбросить пароль в веб-табеле. для самостоятельного сброса пароля позвоните с вашего мобильного телефона на +7 (xxx) xxx xx xx, наберите добавочный номер 10100, нажмите * и подтвердите сброс пароля, нажав #. обновленный пароль отправляется по sms.        249
кнопка "изменить номер" телефона находится в личном разделе в лк. если доступа к лк нет, для смены номера телефона, обратитесь в поддержку                                                                                                                                                                                                                                                                                                                         

In [410]:
df_qa.to_csv('data_qa_processed.csv', index=False)